In [2]:
# ==============================================================================
# SECTION 1: IMPORT LIBRARIES DAN KONFIGURASI AWAL
# ==============================================================================
"""
Section ini mengimpor semua library yang diperlukan untuk simulasi:
- numpy: operasi numerik dan random generation
- pandas: manipulasi data dalam bentuk DataFrame
- scipy.stats: fungsi statistik seperti pearsonr untuk korelasi
- matplotlib & seaborn: visualisasi data
"""

import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from math import sqrt, pi
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Set seed untuk reproduktibilitas hasil
np.random.seed(42)

print("=" * 80)
print("SIMULASI PERBANDINGAN 4 METODE ASESMEN ADAPTIF")
print("=" * 80)
print("\nMetode yang dibandingkan:")
print("1. CAT dengan EAP (Expected A Posteriori)")
print("2. CAT dengan MLE (Maximum Likelihood Estimation)")
print("3. Rule-Based dengan Prior (Bayesian Scoring)")
print("4. Rule-Based tanpa Prior (Direct Weighted Scoring)")
print("=" * 80)


# ==============================================================================
# SECTION 2: KONFIGURASI PARAMETER
# ==============================================================================
"""
Konfigurasi untuk Rule-Based Method:
- lambda_val: parameter smoothing Bayesian (5 = balanced)
- prior: asumsi awal kemampuan siswa (50 dari skala 0-100)
- win_es: window size untuk evaluasi stabilitas (5 soal terakhir)
- delta_thr: threshold stabilitas estimasi skor (2.0 poin)
- min_items: minimum soal yang harus dijawab (20)
- max_items: maksimum soal yang dapat dijawab (40)
- w_cog: bobot untuk setiap level kognitif C1-C6
- w_diff: bobot untuk setiap level kesulitan D1-D3
"""

RB_CONFIG = {
    'lambda_val': 5,
    'prior': 50,
    'win_es': 5,
    'delta_thr': 2.0,
    'min_items': 20,
    'max_items': 40,
    'w_cog': {'C1': 1.0, 'C2': 1.1, 'C3': 1.2, 'C4': 1.3, 'C5': 1.4, 'C6': 1.5},
    'w_diff': {'D1': 1.0, 'D2': 1.2, 'D3': 1.4}
}

"""
Konfigurasi untuk CAT Method:
- se_target: target standard error untuk stopping rule (0.32 ≈ reliability 0.90)
- min_items: minimum soal (20)
- max_items: maksimum soal (40)
- estimator: metode estimasi theta ('EAP' atau 'MLE')
"""

CAT_CONFIG = {
    'se_target': 0.32,
    'min_items': 20,
    'max_items': 40,
    'estimator': 'EAP'  # akan diubah untuk simulasi MLE
}

print("\n✓ Konfigurasi parameter telah dimuat")


# ==============================================================================
# SECTION 3: GENERATE BANK SOAL
# ==============================================================================
"""
Fungsi ini membuat bank soal dengan struktur:
- 6 level kognitif (C1-C6): dari low order ke high order thinking
- 3 level kesulitan (D1-D3): mudah, sedang, sulit
- 56 item per kombinasi C×D = 1008 total items

Parameter IRT 2PL:
- a (discrimination): 0.8-1.5, mengukur seberapa baik item membedakan kemampuan
- b (difficulty): range -2.0 hingga 1.6, disesuaikan dengan C dan D level
  Formula: b = -2.0 + (c_idx × 0.6) + (d_idx × 0.3) + noise
"""

def generate_bank():
    items = []
    id_counter = 1

    for c_idx, c in enumerate(['C1', 'C2', 'C3', 'C4', 'C5', 'C6']):
        for d_idx, d in enumerate(['D1', 'D2', 'D3']):
            for _ in range(56):
                # Difficulty parameter dengan progressive increment
                b = -2.0 + (c_idx * 0.6) + (d_idx * 0.3) + np.random.normal(0, 0.1)

                # Discrimination parameter
                a = np.random.uniform(0.8, 1.5)

                items.append({
                    'id': id_counter,
                    'C': c,
                    'D': d,
                    'a': a,
                    'b': b,
                    'w_cog': RB_CONFIG['w_cog'][c],
                    'w_diff': RB_CONFIG['w_diff'][d]
                })
                id_counter += 1

    return pd.DataFrame(items)

BANK_DF = generate_bank()
print(f"\n✓ Bank soal berhasil dibuat: {len(BANK_DF)} items")
print(f"  - Struktur: 6 Cognitive × 3 Difficulty × 56 items = {len(BANK_DF)}")
print(f"  - Range difficulty (b): [{BANK_DF['b'].min():.2f}, {BANK_DF['b'].max():.2f}]")
print(f"  - Range discrimination (a): [{BANK_DF['a'].min():.2f}, {BANK_DF['a'].max():.2f}]")


# ==============================================================================
# SECTION 4: FUNGSI IRT (Item Response Theory)
# ==============================================================================
"""
Fungsi-fungsi dasar IRT 2-Parameter Logistic Model
"""

def p_correct(theta, a, b):
    """
    Menghitung probabilitas jawaban benar menggunakan IRT 2PL

    Formula: P(θ) = 1 / (1 + exp(-a(θ - b)))

    Parameters:
    - theta: ability parameter siswa
    - a: discrimination parameter item
    - b: difficulty parameter item

    Interpretasi:
    - Jika θ = b: P = 0.5 (50% chance benar)
    - Jika θ > b: P > 0.5 (lebih mudah)
    - Jika θ < b: P < 0.5 (lebih sulit)
    """
    z = a * (theta - b)
    z = np.clip(z, -20, 20)  # prevent overflow
    return 1.0 / (1.0 + np.exp(-z))


def item_information(theta, a, b):
    """
    Menghitung Fisher Information untuk item pada theta tertentu

    Formula: I(θ) = a² × P(θ) × (1 - P(θ))

    Informasi maksimal terjadi ketika θ = b (P = 0.5)
    Digunakan untuk memilih item yang paling informatif dalam CAT
    """
    p = p_correct(theta, a, b)
    return (a**2) * p * (1.0 - p)

print("\n✓ Fungsi IRT telah didefinisikan")


# ==============================================================================
# SECTION 5: ESTIMASI THETA - EAP METHOD
# ==============================================================================
"""
Expected A Posteriori (EAP) - Bayesian approach untuk estimasi theta

Proses:
1. Prior: N(0,1) - asumsi distribusi populasi normal
2. Likelihood: P(responses|θ) dari pola jawaban siswa
3. Posterior: prior × likelihood (Bayes' theorem)
4. EAP estimate: E[θ|responses] = integral dari θ × posterior

Keuntungan EAP:
- Stabil untuk semua pola jawaban (termasuk all correct/incorrect)
- Memberikan estimasi bahkan dengan sedikit item
- Menghasilkan standard error yang reliable
"""

def theta_eap_grid(responses, a_vec, b_vec, grid=np.linspace(-4, 4, 161)):
    """
    Estimasi theta menggunakan EAP dengan numerical integration

    Parameters:
    - responses: array jawaban siswa (0 atau 1)
    - a_vec: array discrimination parameters
    - b_vec: array difficulty parameters
    - grid: grid theta untuk integrasi (-4 to 4, 161 points = resolusi 0.05)

    Returns:
    - theta_eap: estimated ability
    - se: standard error of estimation
    """
    # Prior distribution: N(0,1)
    prior = np.exp(-0.5 * (grid ** 2)) / sqrt(2.0 * pi)

    # Likelihood: P(responses|theta)
    like = np.ones_like(grid, dtype=float)
    for r, a, b in zip(responses, a_vec, b_vec):
        p = 1.0 / (1.0 + np.exp(-a * (grid - b)))
        like *= (p ** r) * ((1.0 - p) ** (1 - r)) + 1e-300

    # Posterior: prior × likelihood
    post = prior * like
    post /= np.trapz(post, grid) + 1e-300  # normalisasi

    # EAP: expected value dari posterior
    theta_eap = float(np.trapz(grid * post, grid))

    # Standard Error: sqrt(variance)
    var = float(np.trapz(((grid - theta_eap) ** 2) * post, grid))
    se = np.sqrt(max(var, 1e-12))

    return theta_eap, se

print("✓ Fungsi EAP telah didefinisikan")


# ==============================================================================
# SECTION 6: ESTIMASI THETA - MLE METHOD
# ==============================================================================
"""
Maximum Likelihood Estimation (MLE) untuk theta

Proses:
1. Mencari theta yang memaksimalkan likelihood function
2. Menggunakan Newton-Raphson iteration
3. Derivative pertama (slope) dan kedua (curvature) dari log-likelihood

Keuntungan MLE:
- Unbiased untuk sampel besar
- Asymptotically efficient

Kelemahan MLE:
- Tidak terdefinisi untuk all correct (theta → ∞) atau all incorrect (theta → -∞)
- Memerlukan variasi dalam response pattern
"""

def theta_mle_newton(responses, a_vec, b_vec, max_iter=50, tol=1e-5):
    """
    Estimasi theta menggunakan MLE dengan Newton-Raphson

    Returns:
    - theta_mle: estimated ability
    - se: standard error (1/sqrt(information))
    """
    responses = np.array(responses, dtype=float)
    a_vec = np.array(a_vec, dtype=float)
    b_vec = np.array(b_vec, dtype=float)

    # Cek edge cases
    if np.all(responses == 1):
        # All correct: gunakan heuristik
        theta_est = np.max(b_vec) + 1.0
        info = np.sum(item_information(theta_est, a_vec, b_vec))
        se = 1.0 / np.sqrt(max(info, 1e-6))
        return theta_est, se

    if np.all(responses == 0):
        # All incorrect: gunakan heuristik
        theta_est = np.min(b_vec) - 1.0
        info = np.sum(item_information(theta_est, a_vec, b_vec))
        se = 1.0 / np.sqrt(max(info, 1e-6))
        return theta_est, se

    # Initial estimate: weighted sum
    theta = np.sum((responses - 0.5) * (a_vec * b_vec)) / np.sum(a_vec**2 + 1e-6)
    theta = np.clip(theta, -4, 4)

    # Newton-Raphson iteration
    for iteration in range(max_iter):
        # Compute probabilities
        z = a_vec * (theta - b_vec)
        z = np.clip(z, -20, 20)
        p = 1.0 / (1.0 + np.exp(-z))

        # First derivative (slope)
        d1 = np.sum(a_vec * (responses - p))

        # Second derivative (curvature)
        d2 = -np.sum((a_vec**2) * p * (1 - p))

        # Newton update
        if abs(d2) < 1e-10:
            break

        delta = d1 / d2
        theta_new = theta - delta
        theta_new = np.clip(theta_new, -4, 4)

        # Check convergence
        if abs(theta_new - theta) < tol:
            theta = theta_new
            break

        theta = theta_new

    # Standard error: 1/sqrt(Fisher Information)
    info = np.sum(item_information(theta, a_vec, b_vec))
    se = 1.0 / np.sqrt(max(info, 1e-6))

    return theta, se

print("✓ Fungsi MLE telah didefinisikan")


# ==============================================================================
# SECTION 7: ITEM SELECTION UNTUK CAT
# ==============================================================================
"""
Maximum Information Criterion untuk CAT

Item yang dipilih adalah item dengan Fisher Information tertinggi
pada estimasi theta saat ini. Ini memastikan setiap item memberikan
informasi maksimal untuk meningkatkan precision estimasi.
"""

def select_by_max_info(theta_est, available_df):
    """
    Memilih item dengan maximum information pada theta saat ini

    Jika ada beberapa item dengan information sama, pilih secara random
    """
    infos = item_information(theta_est, available_df['a'].values, available_df['b'].values)
    max_info = np.max(infos)
    candidates = available_df.iloc[np.where(np.isclose(infos, max_info))[0]]
    return candidates.sample(1).iloc[0]

print("✓ Fungsi item selection telah didefinisikan")


# ==============================================================================
# SECTION 8: SIMULASI CAT DENGAN EAP
# ==============================================================================
"""
Computer Adaptive Testing (CAT) dengan EAP estimator

Algoritma:
1. Mulai dengan theta = 0 (population mean)
2. Pilih item dengan maximum information
3. Simulasikan response berdasarkan true theta
4. Re-estimasi theta menggunakan EAP
5. Ulangi hingga SE ≤ target atau mencapai max items

Stopping Rule:
- Minimum 20 items DAN SE ≤ 0.32 (reliability ~0.90)
- Atau maksimum 40 items tercapai
"""

def run_cat_eap_simulation(theta_true):
    """
    Menjalankan simulasi CAT dengan EAP estimator

    Returns:
    - score_0_100: final score dalam skala 0-100
    - history: list detail setiap item yang diberikan
    """
    theta_est = 0.0  # initial estimate
    used_ids = set()
    history = []
    resp_list, a_list, b_list = [], [], []

    for i in range(CAT_CONFIG['max_items']):
        # Pilih item yang belum pernah diberikan
        available = BANK_DF[~BANK_DF['id'].isin(used_ids)]
        if available.empty:
            break

        item = select_by_max_info(theta_est, available)
        used_ids.add(item['id'])

        # Simulasi response
        p = p_correct(theta_true, item['a'], item['b'])
        is_correct = 1 if np.random.random() < p else 0

        # Update estimasi theta
        resp_list.append(is_correct)
        a_list.append(item['a'])
        b_list.append(item['b'])

        theta_est, se = theta_eap_grid(np.array(resp_list), np.array(a_list), np.array(b_list))

        history.append({
            'q_no': i + 1,
            'item_id': item['id'],
            'C': item['C'],
            'D': item['D'],
            'b_param': item['b'],
            'is_correct': is_correct,
            'theta_est': theta_est,
            'se': se
        })

        # Check stopping rule
        if len(history) >= CAT_CONFIG['min_items'] and se <= CAT_CONFIG['se_target']:
            break

    # Convert theta ke skala 0-100
    score_0_100 = p_correct(theta_est, BANK_DF['a'].values, BANK_DF['b'].values).mean() * 100.0

    return score_0_100, history

print("✓ Fungsi CAT-EAP telah didefinisikan")


# ==============================================================================
# SECTION 9: SIMULASI CAT DENGAN MLE
# ==============================================================================
"""
Computer Adaptive Testing (CAT) dengan MLE estimator

Sama dengan CAT-EAP tetapi menggunakan Maximum Likelihood Estimation
untuk estimasi theta. MLE lebih optimal secara asymptotic tetapi
kurang stabil untuk pola jawaban ekstrem.
"""

def run_cat_mle_simulation(theta_true):
    """
    Menjalankan simulasi CAT dengan MLE estimator

    Returns:
    - score_0_100: final score dalam skala 0-100
    - history: list detail setiap item yang diberikan
    """
    theta_est = 0.0
    used_ids = set()
    history = []
    resp_list, a_list, b_list = [], [], []

    for i in range(CAT_CONFIG['max_items']):
        available = BANK_DF[~BANK_DF['id'].isin(used_ids)]
        if available.empty:
            break

        item = select_by_max_info(theta_est, available)
        used_ids.add(item['id'])

        # Simulasi response
        p = p_correct(theta_true, item['a'], item['b'])
        is_correct = 1 if np.random.random() < p else 0

        # Update dengan MLE
        resp_list.append(is_correct)
        a_list.append(item['a'])
        b_list.append(item['b'])

        theta_est, se = theta_mle_newton(np.array(resp_list), np.array(a_list), np.array(b_list))

        history.append({
            'q_no': i + 1,
            'item_id': item['id'],
            'C': item['C'],
            'D': item['D'],
            'b_param': item['b'],
            'is_correct': is_correct,
            'theta_est': theta_est,
            'se': se
        })

        # Check stopping rule
        if len(history) >= CAT_CONFIG['min_items'] and se <= CAT_CONFIG['se_target']:
            break

    # Convert ke 0-100 scale
    score_0_100 = p_correct(theta_est, BANK_DF['a'].values, BANK_DF['b'].values).mean() * 100.0

    return score_0_100, history

print("✓ Fungsi CAT-MLE telah didefinisikan")


# ==============================================================================
# SECTION 10: ROUTING LOGIC UNTUK RULE-BASED
# ==============================================================================
"""
Routing Logic untuk metode Rule-Based

Aturan routing (slowed rule):
1. Jika BENAR:
   - Jika sudah di D3 (difficulty tertinggi): naik ke cognitive level berikutnya, reset ke D1
   - Jika belum D3: naik difficulty dalam cognitive level yang sama

2. Jika SALAH:
   - Jika sudah di D1 (difficulty terendah): tetap di D1
   - Jika belum D1: turun difficulty

Routing ini lebih "lambat" dalam menaikkan kesulitan dibanding routing agresif,
memberikan kesempatan lebih bagi siswa untuk beradaptasi.
"""

def get_next_routing(c, d, is_correct):
    """
    Menentukan cognitive dan difficulty level berikutnya berdasarkan response

    Parameters:
    - c: current cognitive level (C1-C6)
    - d: current difficulty level (D1-D3)
    - is_correct: boolean, apakah jawaban benar

    Returns:
    - next_c, next_d: level berikutnya
    """
    c_levels = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']
    d_levels = ['D1', 'D2', 'D3']
    c_idx = c_levels.index(c)
    d_idx = d_levels.index(d)

    if is_correct:
        if d == 'D3':
            # Sudah di difficulty tertinggi, naik cognitive
            if c_idx < 5:
                return c_levels[c_idx + 1], 'D1'
            else:
                return 'C6', 'D3'  # sudah maksimal
        else:
            # Naik difficulty
            return c, d_levels[d_idx + 1]
    else:
        # Turun difficulty
        if d == 'D1':
            return c, 'D1'  # sudah minimal
        else:
            return c, d_levels[d_idx - 1]

print("✓ Fungsi routing logic telah didefinisikan")


# ==============================================================================
# SECTION 11: SIMULASI RULE-BASED DENGAN PRIOR (BAYESIAN SCORING)
# ==============================================================================
"""
Rule-Based Method dengan Prior (Bayesian Scoring)

Scoring Formula:
ES = (λ × prior + 100 × Σw_correct) / (λ + Σw_attempted)

Dimana:
- λ (lambda): smoothing parameter (5)
- prior: prior score (50)
- w_correct: sum of weights dari jawaban benar
- w_attempted: sum of weights dari semua jawaban
- weight = w_cog × w_diff

Stopping Rule:
- Minimum 20 items DAN estimasi skor stabil
- Stabilitas: range ES dalam 5 soal terakhir ≤ 2.0
- Atau maksimum 40 items

Bayesian scoring memberikan shrinkage terhadap prior, membuat
estimasi lebih stabil terutama di awal tes.
"""

def run_rb_with_prior_simulation(theta_true):
    """
    Simulasi Rule-Based dengan Bayesian scoring (dengan prior)

    Returns:
    - final_score: skor akhir dalam skala 0-100
    - history: detail setiap item
    """
    history = []
    curr_c, curr_d = 'C1', 'D1'  # starting position
    total_w_attempted = 0
    total_w_correct = 0
    es_history = []
    answered_ids = []
    stability_reached = False

    for i in range(RB_CONFIG['max_items']):
        # Pilih item dari current cell (C, D)
        pool = BANK_DF[
            (BANK_DF['C'] == curr_c) &
            (BANK_DF['D'] == curr_d) &
            (~BANK_DF['id'].isin(answered_ids))
        ]

        # Jika cell kosong, pilih dari pool manapun
        if pool.empty:
            pool = BANK_DF[~BANK_DF['id'].isin(answered_ids)]
        if pool.empty:
            break

        item = pool.sample(1).iloc[0]
        answered_ids.append(item['id'])

        # Simulasi response
        p = p_correct(theta_true, item['a'], item['b'])
        is_correct = 1 if np.random.random() < p else 0

        # Update weighted score
        weight = item['w_cog'] * item['w_diff']
        total_w_attempted += weight
        if is_correct:
            total_w_correct += weight

        # Bayesian scoring dengan prior
        es = ((RB_CONFIG['lambda_val'] * RB_CONFIG['prior']) + (100 * total_w_correct)) / \
             (RB_CONFIG['lambda_val'] + total_w_attempted)
        es_history.append(es)

        history.append({
            'q_no': i + 1,
            'item_id': item['id'],
            'C': curr_c,
            'D': curr_d,
            'b_param': item['b'],
            'is_correct': is_correct,
            'es': es
        })

        # Check stabilitas
        n = len(es_history)
        if n >= RB_CONFIG['win_es']:
            window = es_history[-RB_CONFIG['win_es']:]
            es_range = max(window) - min(window)
            if es_range <= RB_CONFIG['delta_thr']:
                stability_reached = True

        # Stopping rule
        if n >= RB_CONFIG['min_items'] and stability_reached:
            break

        # Routing untuk item selanjutnya
        curr_c, curr_d = get_next_routing(curr_c, curr_d, is_correct)

    final_score = es_history[-1] if es_history else RB_CONFIG['prior']

    return final_score, history

print("✓ Fungsi Rule-Based dengan Prior telah didefinisikan")


# ==============================================================================
# SECTION 12: SIMULASI RULE-BASED TANPA PRIOR (DIRECT WEIGHTED SCORING)
# ==============================================================================
"""
Rule-Based Method tanpa Prior (Direct Weighted Scoring)

Scoring Formula:
ES = (100 × Σw_correct) / Σw_attempted

Tidak ada shrinkage terhadap prior, estimasi murni dari performance.
Ini membuat scoring lebih "volatile" terutama di awal tes, tetapi
lebih responsive terhadap performa aktual siswa.

Stopping Rule: sama dengan RB with prior
"""

def run_rb_without_prior_simulation(theta_true):
    """
    Simulasi Rule-Based tanpa prior (direct weighted scoring)

    Returns:
    - final_score: skor akhir dalam skala 0-100
    - history: detail setiap item
    """
    history = []
    curr_c, curr_d = 'C1', 'D1'
    total_w_attempted = 0
    total_w_correct = 0
    es_history = []
    answered_ids = []
    stability_reached = False

    for i in range(RB_CONFIG['max_items']):
        pool = BANK_DF[
            (BANK_DF['C'] == curr_c) &
            (BANK_DF['D'] == curr_d) &
            (~BANK_DF['id'].isin(answered_ids))
        ]

        if pool.empty:
            pool = BANK_DF[~BANK_DF['id'].isin(answered_ids)]
        if pool.empty:
            break

        item = pool.sample(1).iloc[0]
        answered_ids.append(item['id'])

        # Simulasi response
        p = p_correct(theta_true, item['a'], item['b'])
        is_correct = 1 if np.random.random() < p else 0

        # Update weighted score
        weight = item['w_cog'] * item['w_diff']
        total_w_attempted += weight
        if is_correct:
            total_w_correct += weight

        # Direct weighted scoring (TANPA prior)
        es = (100 * total_w_correct) / total_w_attempted if total_w_attempted > 0 else 0
        es_history.append(es)

        history.append({
            'q_no': i + 1,
            'item_id': item['id'],
            'C': curr_c,
            'D': curr_d,
            'b_param': item['b'],
            'is_correct': is_correct,
            'es': es
        })

        # Check stabilitas
        n = len(es_history)
        if n >= RB_CONFIG['win_es']:
            window = es_history[-RB_CONFIG['win_es']:]
            es_range = max(window) - min(window)
            if es_range <= RB_CONFIG['delta_thr']:
                stability_reached = True

        # Stopping rule
        if n >= RB_CONFIG['min_items'] and stability_reached:
            break

        # Routing
        curr_c, curr_d = get_next_routing(curr_c, curr_d, is_correct)

    final_score = es_history[-1] if es_history else 0

    return final_score, history

print("✓ Fungsi Rule-Based tanpa Prior telah didefinisikan")


# ==============================================================================
# SECTION 13: GENERATE TRUE THETA DAN TRUE SCORE
# ==============================================================================
"""
Generate populasi siswa dengan distribusi theta normal

N = 1000 siswa
Theta ~ N(0, 1)
- Mean 0: rata-rata populasi
- SD 1: standar deviation normal

True score dihitung sebagai expected proportion correct pada semua items
dalam bank soal, kemudian dikonversi ke skala 0-100.
"""

N_STUDENTS = 1000
thetas = np.random.normal(0, 1, N_STUDENTS)

print(f"\n✓ Generated {N_STUDENTS} siswa")
print(f"  Theta range: [{thetas.min():.2f}, {thetas.max():.2f}]")
print(f"  Theta mean: {thetas.mean():.2f}, SD: {thetas.std():.2f}")


# ==============================================================================
# SECTION 14: MENJALANKAN SEMUA SIMULASI
# ==============================================================================
"""
Menjalankan 4 metode simulasi untuk setiap siswa:
1. CAT-EAP
2. CAT-MLE
3. Rule-Based dengan Prior
4. Rule-Based tanpa Prior

Untuk setiap siswa dan metode, kita simpan:
- student_id
- method
- theta (true ability)
- true_score (expected score)
- score (estimated score dari metode)
- items (jumlah soal yang dijawab)
- history (detail setiap item)
"""

results = []
print("\n" + "=" * 80)
print("MENJALANKAN SIMULASI...")
print("=" * 80)

for idx, theta in enumerate(thetas):
    if (idx + 1) % 250 == 0:
        print(f"Progress: {idx + 1}/{N_STUDENTS} siswa ({(idx+1)/N_STUDENTS*100:.1f}%)")

    # Hitung true score (expected proportion correct)
    true_score = p_correct(theta, BANK_DF['a'].values, BANK_DF['b'].values).mean() * 100

    # 1. CAT dengan EAP
    cat_eap_score, cat_eap_hist = run_cat_eap_simulation(theta)
    results.append({
        'student_id': idx,
        'method': 'CAT-EAP',
        'theta': theta,
        'true_score': true_score,
        'score': cat_eap_score,
        'items': len(cat_eap_hist),
        'history': cat_eap_hist
    })

    # 2. CAT dengan MLE
    cat_mle_score, cat_mle_hist = run_cat_mle_simulation(theta)
    results.append({
        'student_id': idx,
        'method': 'CAT-MLE',
        'theta': theta,
        'true_score': true_score,
        'score': cat_mle_score,
        'items': len(cat_mle_hist),
        'history': cat_mle_hist
    })

    # 3. Rule-Based dengan Prior
    rb_prior_score, rb_prior_hist = run_rb_with_prior_simulation(theta)
    results.append({
        'student_id': idx,
        'method': 'RB-Prior',
        'theta': theta,
        'true_score': true_score,
        'score': rb_prior_score,
        'items': len(rb_prior_hist),
        'history': rb_prior_hist
    })

    # 4. Rule-Based tanpa Prior
    rb_no_prior_score, rb_no_prior_hist = run_rb_without_prior_simulation(theta)
    results.append({
        'student_id': idx,
        'method': 'RB-NoPrior',
        'theta': theta,
        'true_score': true_score,
        'score': rb_no_prior_score,
        'items': len(rb_no_prior_hist),
        'history': rb_no_prior_hist
    })

df_results = pd.DataFrame(results)
print(f"\n✓ Simulasi selesai: {len(df_results)} records")
print(f"  {N_STUDENTS} siswa × 4 metode = {len(df_results)} total simulasi")


# ==============================================================================
# SECTION 15: CLUSTERING BERDASARKAN THETA
# ==============================================================================
"""
Membagi siswa menjadi 5 kelompok berdasarkan true theta menggunakan quintiles:
1. Sangat Rendah: bottom 20%
2. Rendah: 20-40%
3. Sedang: 40-60%
4. Tinggi: 60-80%
5. Sangat Tinggi: top 20%

Clustering ini membantu kita menganalisis performa metode pada
berbagai level kemampuan siswa.
"""

df_students = df_results[['student_id', 'theta']].drop_duplicates()
labels_5 = ['Sangat Rendah', 'Rendah', 'Sedang', 'Tinggi', 'Sangat Tinggi']

df_students['group_theta'] = pd.qcut(df_students['theta'], 5, labels=labels_5)
df_results = df_results.merge(df_students[['student_id', 'group_theta']], on='student_id')

print("\n" + "=" * 80)
print("CLUSTERING BERDASARKAN THETA")
print("=" * 80)
print(df_students.groupby('group_theta', observed=True)['theta'].agg(['count', 'min', 'max', 'mean']))


# ==============================================================================
# SECTION 16: CLUSTERING BERDASARKAN SKOR AKHIR
# ==============================================================================
"""
Untuk setiap metode, bagi siswa berdasarkan skor akhir mereka.
Ini membantu kita melihat apakah metode menghasilkan distribusi
skor yang reasonable.
"""

for method in ['CAT-EAP', 'CAT-MLE', 'RB-Prior', 'RB-NoPrior']:
    subset = df_results[df_results['method'] == method].copy()
    try:
        score_groups = pd.qcut(subset['score'], 5, labels=labels_5, duplicates='drop')
    except ValueError:
        score_groups = pd.cut(subset['score'], 5, labels=labels_5)

    df_results.loc[df_results['method'] == method, 'group_score'] = score_groups

print("\n✓ Clustering berdasarkan skor selesai")


# ==============================================================================
# SECTION 17: SUMMARY PERBANDINGAN GLOBAL
# ==============================================================================
"""
Metrik evaluasi global untuk setiap metode:

1. Correlation (r): korelasi Pearson antara estimated score dan true score
   - Range: -1 to 1
   - Nilai lebih tinggi = prediksi lebih akurat
   - r > 0.9 = excellent

2. RMSE: Root Mean Square Error
   - Range: 0 to ∞
   - Nilai lebih rendah = error lebih kecil
   - Mengukur magnitude rata-rata error

3. Bias: Mean error (estimated - true)
   - Range: -∞ to ∞
   - Nilai mendekati 0 = unbiased
   - Positif = overestimate, Negatif = underestimate

4. Avg Items: rata-rata jumlah soal yang dijawab
   - Range: min_items to max_items
   - Lebih sedikit = lebih efisien (jika akurasi tetap baik)
"""

summary_rows = []
methods = ['CAT-EAP', 'CAT-MLE', 'RB-Prior', 'RB-NoPrior']

for method in methods:
    subset = df_results[df_results['method'] == method]

    r_val, p_val = pearsonr(subset['score'], subset['true_score'])
    rmse_val = np.sqrt(np.mean((subset['score'] - subset['true_score'])**2))
    bias_val = np.mean(subset['score'] - subset['true_score'])
    avg_items = subset['items'].mean()

    summary_rows.append({
        'method': method,
        'correlation': r_val,
        'rmse': rmse_val,
        'bias': bias_val,
        'avg_items': avg_items
    })

df_summary = pd.DataFrame(summary_rows)

print("\n" + "=" * 80)
print("SUMMARY PERBANDINGAN GLOBAL")
print("=" * 80)
print(df_summary.to_string(index=False))
print("\nInterpretasi:")
print("- Correlation: semakin tinggi semakin baik (>0.9 excellent)")
print("- RMSE: semakin rendah semakin baik (error lebih kecil)")
print("- Bias: mendekati 0 lebih baik (unbiased)")
print("- Avg Items: efisiensi (lebih sedikit lebih baik jika akurasi tetap)")


# ==============================================================================
# SECTION 18: SUMMARY PER KELOMPOK THETA
# ==============================================================================
"""
Analisis performa per kelompok kemampuan

Ini penting untuk melihat apakah metode:
- Overestimate siswa lemah (positive bias di kelompok rendah)
- Underestimate siswa pandai (negative bias di kelompok tinggi)
- Konsisten akurat di semua level
"""

group_summary = []

for method in methods:
    subset = df_results[df_results['method'] == method]
    for g in labels_5:
        subg = subset[subset['group_theta'] == g]

        if len(subg) < 2:
            continue

        r_val, _ = pearsonr(subg['score'], subg['true_score'])
        rmse_val = np.sqrt(np.mean((subg['score'] - subg['true_score'])**2))
        bias_val = np.mean(subg['score'] - subg['true_score'])
        avg_items = subg['items'].mean()

        group_summary.append({
            'method': method,
            'group_theta': g,
            'correlation': r_val,
            'rmse': rmse_val,
            'bias': bias_val,
            'avg_items': avg_items
        })

df_group_summary = pd.DataFrame(group_summary)

print("\n" + "=" * 80)
print("SUMMARY PER KELOMPOK THETA")
print("=" * 80)
print(df_group_summary.to_string(index=False))


# ==============================================================================
# SECTION 19: DISTRIBUSI KOGNITIF
# ==============================================================================
"""
Menganalisis distribusi level kognitif (C1-C6) yang diberikan
oleh setiap metode.

Ini menunjukkan:
- Apakah CAT memberikan lebih banyak soal high-order thinking (C4-C6)?
- Apakah Rule-Based cenderung stuck di level tertentu?
- Balance antara low dan high order thinking
"""

cognitive_rows = []

for method in methods:
    subset = df_results[df_results['method'] == method]

    # Expand history
    expanded = []
    for _, row in subset.iterrows():
        for h in row['history']:
            expanded.append({
                'method': method,
                'C': h['C'],
                'D': h['D']
            })

    df_exp = pd.DataFrame(expanded)
    total = len(df_exp)

    # Hitung proporsi per cognitive level
    for c in ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']:
        count = (df_exp['C'] == c).sum()
        prop = count / total if total > 0 else 0
        cognitive_rows.append({
            'method': method,
            'C_level': c,
            'count': count,
            'proportion': prop
        })

df_cognitive = pd.DataFrame(cognitive_rows)

print("\n" + "=" * 80)
print("DISTRIBUSI LEVEL KOGNITIF (C1-C6)")
print("=" * 80)
for method in methods:
    print(f"\n{method}:")
    subset = df_cognitive[df_cognitive['method'] == method]
    for _, row in subset.iterrows():
        print(f"  {row['C_level']}: {row['proportion']*100:5.2f}% ({row['count']:5d} items)")


# ==============================================================================
# SECTION 20: DISTRIBUSI DIFFICULTY
# ==============================================================================
"""
Menganalisis distribusi level difficulty (D1-D3) yang diberikan
oleh setiap metode.
"""

difficulty_rows = []

for method in methods:
    subset = df_results[df_results['method'] == method]

    expanded = []
    for _, row in subset.iterrows():
        for h in row['history']:
            expanded.append({
                'method': method,
                'D': h['D']
            })

    df_exp = pd.DataFrame(expanded)
    total = len(df_exp)

    for d in ['D1', 'D2', 'D3']:
        count = (df_exp['D'] == d).sum()
        prop = count / total if total > 0 else 0
        difficulty_rows.append({
            'method': method,
            'D_level': d,
            'count': count,
            'proportion': prop
        })

df_difficulty = pd.DataFrame(difficulty_rows)

print("\n" + "=" * 80)
print("DISTRIBUSI LEVEL DIFFICULTY (D1-D3)")
print("=" * 80)
for method in methods:
    print(f"\n{method}:")
    subset = df_difficulty[df_difficulty['method'] == method]
    for _, row in subset.iterrows():
        print(f"  {row['D_level']}: {row['proportion']*100:5.2f}% ({row['count']:5d} items)")


# ==============================================================================
# SECTION 21: PILIH 5 SISWA RANDOM UNTUK DETAIL ANALYSIS
# ==============================================================================
"""
Memilih 5 siswa secara random (1 dari setiap kelompok theta)
untuk analisis detail pola jawaban dan estimasi skor.
"""

selected_students = []
for group in labels_5:
    students_in_group = df_students[df_students['group_theta'] == group]['student_id'].values
    if len(students_in_group) > 0:
        selected = np.random.choice(students_in_group, 1)[0]
        selected_students.append(selected)

print("\n" + "=" * 80)
print("SISWA TERPILIH UNTUK DETAIL ANALYSIS")
print("=" * 80)
for sid in selected_students:
    student_info = df_students[df_students['student_id'] == sid].iloc[0]
    print(f"Student ID {sid}: Theta = {student_info['theta']:.3f}, Group = {student_info['group_theta']}")

# Export detail untuk 5 siswa ini
detail_records = []

for sid in selected_students:
    student_info = df_students[df_students['student_id'] == sid].iloc[0]

    for method in methods:
        record = df_results[(df_results['student_id'] == sid) & (df_results['method'] == method)].iloc[0]

        for item_data in record['history']:
            detail_records.append({
                'student_id': sid,
                'theta': record['theta'],
                'true_score': record['true_score'],
                'group_theta': student_info['group_theta'],
                'method': method,
                'final_score': record['score'],
                'total_items': record['items'],
                'q_no': item_data['q_no'],
                'item_id': item_data.get('item_id', ''),
                'C_level': item_data['C'],
                'D_level': item_data['D'],
                'b_param': item_data.get('b_param', ''),
                'is_correct': item_data['is_correct'],
                'theta_est': item_data.get('theta_est', ''),
                'se': item_data.get('se', ''),
                'es': item_data.get('es', '')
            })

df_detail = pd.DataFrame(detail_records)


# ==============================================================================
# SECTION 22: EXPORT SEMUA DATA KE CSV
# ==============================================================================
"""
Export berbagai tabel hasil ke CSV untuk analisis lebih lanjut
"""

print("\n" + "=" * 80)
print("EXPORT DATA KE CSV")
print("=" * 80)

# 1. Full results
df_results_export = df_results.drop(columns=['history'])
df_results_export.to_csv("results_full.csv", index=False)
print("✓ results_full.csv - Hasil lengkap semua siswa dan metode")

# 2. Summary global
df_summary.to_csv("summary_global.csv", index=False)
print("✓ summary_global.csv - Metrik global per metode")

# 3. Summary per kelompok
df_group_summary.to_csv("summary_per_kelompok.csv", index=False)
print("✓ summary_per_kelompok.csv - Metrik per kelompok theta")

# 4. Distribusi kognitif
df_cognitive.to_csv("distribusi_kognitif.csv", index=False)
print("✓ distribusi_kognitif.csv - Distribusi level C1-C6")

# 5. Distribusi difficulty
df_difficulty.to_csv("distribusi_difficulty.csv", index=False)
print("✓ distribusi_difficulty.csv - Distribusi level D1-D3")

# 6. Detail 5 siswa
df_detail.to_csv("detail_5_siswa.csv", index=False)
print("✓ detail_5_siswa.csv - Detail item-by-item untuk 5 siswa terpilih")

# 7. Info siswa
df_students.to_csv("info_siswa.csv", index=False)
print("✓ info_siswa.csv - Informasi theta dan kelompok semua siswa")


# ==============================================================================
# SECTION 23: VISUALISASI - DISTRIBUSI KOGNITIF PER KELOMPOK
# ==============================================================================
"""
Membuat visualisasi distribusi kognitif untuk setiap metode
di setiap kelompok kemampuan (clustering berdasarkan theta)
"""

def plot_cognitive_by_group(data_results, group_col='group_theta'):
    """
    Membuat facet plot distribusi kognitif
    """
    # Expand history
    expanded = []
    for _, row in data_results.iterrows():
        for h in row['history']:
            expanded.append({
                'method': row['method'],
                'group': row[group_col],
                'C': h['C']
            })

    df_plot = pd.DataFrame(expanded)

    # Hitung persentase
    df_count = df_plot.groupby(['method', 'group', 'C']).size().reset_index(name='count')
    df_total = df_plot.groupby(['method', 'group']).size().reset_index(name='total')
    df_merged = df_count.merge(df_total, on=['method', 'group'])
    df_merged['percentage'] = (df_merged['count'] / df_merged['total']) * 100

    # Plot
    g = sns.FacetGrid(
        df_merged,
        row='method',
        col='group',
        col_order=labels_5,
        height=3,
        aspect=0.8,
        sharex=True,
        sharey=True
    )

    g.map_dataframe(
        sns.barplot,
        x='C',
        y='percentage',
        order=['C1', 'C2', 'C3', 'C4', 'C5', 'C6'],
        palette='viridis',
        edgecolor='black',
        linewidth=0.5
    )

    for ax in g.axes.flat:
        ax.grid(axis='y', linestyle='--', alpha=0.3)
        ax.set_ylim(0, 60)

    g.set_axis_labels("Level Kognitif", "Persentase (%)")
    g.set_titles(col_template="{col_name}", row_template="{row_name}")
    plt.subplots_adjust(top=0.93)
    g.fig.suptitle(f"Distribusi Kognitif per Kelompok ({group_col.replace('_', ' ').title()})",
                   fontsize=14, fontweight='bold')

    return g

print("\n" + "=" * 80)
print("MEMBUAT VISUALISASI")
print("=" * 80)

# Plot 1: Distribusi kognitif per kelompok theta
g1 = plot_cognitive_by_group(df_results, 'group_theta')
plt.savefig("viz_kognitif_per_theta.png", dpi=300, bbox_inches='tight')
print("✓ viz_kognitif_per_theta.png")
plt.close()

# Plot 2: Distribusi kognitif per kelompok score
g2 = plot_cognitive_by_group(df_results, 'group_score')
plt.savefig("viz_kognitif_per_score.png", dpi=300, bbox_inches='tight')
print("✓ viz_kognitif_per_score.png")
plt.close()


# ==============================================================================
# SECTION 24: VISUALISASI - RATA-RATA JUMLAH SOAL
# ==============================================================================
"""
Visualisasi efisiensi: berapa banyak soal yang dijawab per kelompok
"""

def plot_avg_items(data_results, group_col='group_theta'):
    """
    Barplot rata-rata jumlah soal per metode dan kelompok
    """
    df_agg = data_results.groupby(['method', group_col])['items'].mean().reset_index()

    g = sns.FacetGrid(df_agg, row='method', height=4, aspect=2.5)

    g.map_dataframe(
        sns.barplot,
        x=group_col,
        y='items',
        order=labels_5,
        palette='coolwarm',
        edgecolor='black',
        linewidth=0.8
    )

    for ax in g.axes.flat:
        ax.grid(axis='y', linestyle='--', alpha=0.3)
        ax.set_ylim(0, 45)
        ax.axhline(y=20, color='green', linestyle='--', linewidth=1, alpha=0.5, label='Min Items')
        ax.axhline(y=40, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Max Items')
        ax.set_ylabel("Rata-rata Jumlah Soal")
        ax.set_xlabel("Kelompok Kemampuan")

        # Annotate bars
        for p in ax.patches:
            height = p.get_height()
            if height > 0:
                ax.text(p.get_x() + p.get_width()/2., height + 0.5,
                       f'{height:.1f}', ha="center", fontsize=9)

    g.set_titles(row_template="{row_name}")
    plt.subplots_adjust(top=0.92)
    g.fig.suptitle(f"Efisiensi: Rata-rata Jumlah Soal ({group_col.replace('_', ' ').title()})",
                   fontsize=14, fontweight='bold')

    return g

# Plot 3: Avg items per theta group
g3 = plot_avg_items(df_results, 'group_theta')
plt.savefig("viz_avg_items_per_theta.png", dpi=300, bbox_inches='tight')
print("✓ viz_avg_items_per_theta.png")
plt.close()

# Plot 4: Avg items per score group
g4 = plot_avg_items(df_results, 'group_score')
plt.savefig("viz_avg_items_per_score.png", dpi=300, bbox_inches='tight')
print("✓ viz_avg_items_per_score.png")
plt.close()


# ==============================================================================
# SECTION 25: VISUALISASI - SCATTER PLOT ACCURACY
# ==============================================================================
"""
Scatter plot: True Score vs Estimated Score untuk setiap metode
"""

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, method in enumerate(methods):
    ax = axes[idx]
    subset = df_results[df_results['method'] == method]

    # Scatter plot
    ax.scatter(subset['true_score'], subset['score'], alpha=0.3, s=20, color='steelblue')

    # Perfect prediction line
    ax.plot([0, 100], [0, 100], 'r--', linewidth=2, label='Perfect Prediction')

    # Regression line
    z = np.polyfit(subset['true_score'], subset['score'], 1)
    p = np.poly1d(z)
    ax.plot(subset['true_score'], p(subset['true_score']),
            'g-', linewidth=2, alpha=0.8, label=f'Fit: y={z[0]:.2f}x+{z[1]:.2f}')

    # Metrics
    r_val = subset[['true_score', 'score']].corr().iloc[0, 1]
    rmse_val = np.sqrt(np.mean((subset['score'] - subset['true_score'])**2))

    ax.set_xlabel('True Score', fontsize=11)
    ax.set_ylabel('Estimated Score', fontsize=11)
    ax.set_title(f'{method}\nr={r_val:.3f}, RMSE={rmse_val:.2f}',
                 fontsize=12, fontweight='bold')
    ax.legend(loc='upper left', fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)

plt.tight_layout()
plt.savefig("viz_accuracy_scatter.png", dpi=300, bbox_inches='tight')
print("✓ viz_accuracy_scatter.png")
plt.close()


# ==============================================================================
# SECTION 26: VISUALISASI - COMPARISON BOXPLOT PER KELOMPOK
# ==============================================================================
"""
Boxplot untuk membandingkan distribusi error di setiap kelompok theta
"""

df_results['error'] = df_results['score'] - df_results['true_score']

fig, axes = plt.subplots(1, 4, figsize=(18, 5), sharey=True)

for idx, method in enumerate(methods):
    ax = axes[idx]
    subset = df_results[df_results['method'] == method]

    sns.boxplot(data=subset, x='group_theta', y='error', order=labels_5,
                palette='Set2', ax=ax)

    ax.axhline(y=0, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
    ax.set_xlabel('Kelompok Theta', fontsize=10)
    ax.set_ylabel('Error (Est - True)' if idx == 0 else '', fontsize=10)
    ax.set_title(method, fontsize=12, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(-40, 40)

plt.tight_layout()
plt.savefig("viz_error_boxplot.png", dpi=300, bbox_inches='tight')
print("✓ viz_error_boxplot.png")
plt.close()


# ==============================================================================
# SECTION 27: PRINT SUMMARY TABLE DETAIL 5 SISWA
# ==============================================================================
"""
Menampilkan summary untuk 5 siswa terpilih
"""

print("\n" + "=" * 80)
print("DETAIL ANALYSIS: 5 SISWA TERPILIH")
print("=" * 80)

for sid in selected_students:
    student_data = df_detail[df_detail['student_id'] == sid]
    if student_data.empty:
        continue

    first_row = student_data.iloc[0]
    print(f"\n{'='*80}")
    print(f"STUDENT ID: {sid}")
    print(f"True Theta: {first_row['theta']:.3f}")
    print(f"True Score: {first_row['true_score']:.2f}")
    print(f"Group: {first_row['group_theta']}")
    print(f"{'='*80}")

    for method in methods:
        method_data = student_data[student_data['method'] == method]
        if method_data.empty:
            continue

        final_score = method_data.iloc[0]['final_score']
        total_items = method_data.iloc[0]['total_items']
        n_correct = method_data['is_correct'].sum()
        accuracy = n_correct / total_items * 100

        print(f"\n{method}:")
        print(f"  Final Score: {final_score:.2f}")
        print(f"  Total Items: {total_items}")
        print(f"  Accuracy: {accuracy:.1f}% ({n_correct}/{total_items})")
        print(f"  Error: {final_score - first_row['true_score']:.2f}")

        # Ringkasan pola jawaban per cognitive level
        c_summary = method_data.groupby('C_level')['is_correct'].agg(['count', 'sum']).reset_index()
        c_summary['accuracy'] = c_summary['sum'] / c_summary['count'] * 100
        print(f"  Cognitive Distribution:")
        for _, row in c_summary.iterrows():
            print(f"    {row['C_level']}: {row['count']} items, {row['accuracy']:.1f}% correct")


# ==============================================================================
# SECTION 28: FINAL SUMMARY DAN KESIMPULAN
# ==============================================================================

print("\n" + "=" * 80)
print("SIMULASI SELESAI!")
print("=" * 80)
print("\nFile yang dihasilkan:")
print("  CSV Files:")
print("    - results_full.csv")
print("    - summary_global.csv")
print("    - summary_per_kelompok.csv")
print("    - distribusi_kognitif.csv")
print("    - distribusi_difficulty.csv")
print("    - detail_5_siswa.csv")
print("    - info_siswa.csv")
print("\n  Visualization Files:")
print("    - viz_kognitif_per_theta.png")
print("    - viz_kognitif_per_score.png")
print("    - viz_avg_items_per_theta.png")
print("    - viz_avg_items_per_score.png")
print("    - viz_accuracy_scatter.png")
print("    - viz_error_boxplot.png")
print("\n" + "=" * 80)
print("Terima kasih telah menggunakan simulasi ini!")
print("=" * 80)


# ==============================================================================
# EXPORT DATA UNTUK SEMUA GRAFIK KE CSV
# ==============================================================================

print("\n" + "=" * 80)
print("EXPORT DATA AGREGAT UNTUK GRAFIK")
print("=" * 80)

# ==============================================================================
# 1. DATA UNTUK GRAFIK DISTRIBUSI KOGNITIF PER KELOMPOK THETA
# ==============================================================================
"""
Format: method | group_theta | C_level | count | percentage
Digunakan untuk: viz_kognitif_per_theta.png
"""

cognitive_theta_data = []

for method in methods:
    subset = df_results[df_results['method'] == method]

    for group in labels_5:
        group_subset = subset[subset['group_theta'] == group]

        # Expand history untuk kelompok ini
        expanded = []
        for _, row in group_subset.iterrows():
            for h in row['history']:
                expanded.append(h['C'])

        total = len(expanded)

        # Hitung per C level
        for c in ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']:
            count = expanded.count(c)
            pct = (count / total * 100) if total > 0 else 0

            cognitive_theta_data.append({
                'method': method,
                'group_theta': group,
                'C_level': c,
                'count': count,
                'percentage': pct
            })

df_cognitive_theta = pd.DataFrame(cognitive_theta_data)
df_cognitive_theta.to_csv("graph_data_cognitive_per_theta.csv", index=False)
print("✓ graph_data_cognitive_per_theta.csv")
print(f"  Rows: {len(df_cognitive_theta)} (4 methods × 5 groups × 6 C-levels)")


# ==============================================================================
# 2. DATA UNTUK GRAFIK DISTRIBUSI KOGNITIF PER KELOMPOK SKOR
# ==============================================================================
"""
Format: method | group_score | C_level | count | percentage
Digunakan untuk: viz_kognitif_per_score.png
"""

cognitive_score_data = []

for method in methods:
    subset = df_results[df_results['method'] == method]

    for group in labels_5:
        group_subset = subset[subset['group_score'] == group]

        if len(group_subset) == 0:
            continue

        # Expand history
        expanded = []
        for _, row in group_subset.iterrows():
            for h in row['history']:
                expanded.append(h['C'])

        total = len(expanded)

        for c in ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']:
            count = expanded.count(c)
            pct = (count / total * 100) if total > 0 else 0

            cognitive_score_data.append({
                'method': method,
                'group_score': group,
                'C_level': c,
                'count': count,
                'percentage': pct
            })

df_cognitive_score = pd.DataFrame(cognitive_score_data)
df_cognitive_score.to_csv("graph_data_cognitive_per_score.csv", index=False)
print("✓ graph_data_cognitive_per_score.csv")
print(f"  Rows: {len(df_cognitive_score)}")


# ==============================================================================
# 3. DATA UNTUK GRAFIK RATA-RATA JUMLAH SOAL PER KELOMPOK THETA
# ==============================================================================
"""
Format: method | group_theta | avg_items | std_items | min_items | max_items
Digunakan untuk: viz_avg_items_per_theta.png
"""

items_theta_data = []

for method in methods:
    subset = df_results[df_results['method'] == method]

    for group in labels_5:
        group_subset = subset[subset['group_theta'] == group]

        items_theta_data.append({
            'method': method,
            'group_theta': group,
            'avg_items': group_subset['items'].mean(),
            'std_items': group_subset['items'].std(),
            'min_items': group_subset['items'].min(),
            'max_items': group_subset['items'].max(),
            'n_students': len(group_subset)
        })

df_items_theta = pd.DataFrame(items_theta_data)
df_items_theta.to_csv("graph_data_items_per_theta.csv", index=False)
print("✓ graph_data_items_per_theta.csv")
print(f"  Rows: {len(df_items_theta)} (4 methods × 5 groups)")


# ==============================================================================
# 4. DATA UNTUK GRAFIK RATA-RATA JUMLAH SOAL PER KELOMPOK SKOR
# ==============================================================================
"""
Format: method | group_score | avg_items | std_items | min_items | max_items
Digunakan untuk: viz_avg_items_per_score.png
"""

items_score_data = []

for method in methods:
    subset = df_results[df_results['method'] == method]

    for group in labels_5:
        group_subset = subset[subset['group_score'] == group]

        if len(group_subset) == 0:
            continue

        items_score_data.append({
            'method': method,
            'group_score': group,
            'avg_items': group_subset['items'].mean(),
            'std_items': group_subset['items'].std(),
            'min_items': group_subset['items'].min(),
            'max_items': group_subset['items'].max(),
            'n_students': len(group_subset)
        })

df_items_score = pd.DataFrame(items_score_data)
df_items_score.to_csv("graph_data_items_per_score.csv", index=False)
print("✓ graph_data_items_per_score.csv")
print(f"  Rows: {len(df_items_score)}")


# ==============================================================================
# 5. DATA UNTUK GRAFIK SCATTER PLOT (True vs Estimated Score)
# ==============================================================================
"""
Format: method | student_id | theta | group_theta | true_score | estimated_score | error
Digunakan untuk: viz_accuracy_scatter.png

Note: Data ini sebenarnya sudah ada di results_full.csv,
tapi kita buat versi simplified tanpa kolom history yang besar
"""

scatter_data = df_results[['method', 'student_id', 'theta', 'group_theta',
                            'true_score', 'score', 'items']].copy()
scatter_data['error'] = scatter_data['score'] - scatter_data['true_score']
scatter_data = scatter_data.rename(columns={'score': 'estimated_score'})

scatter_data.to_csv("graph_data_scatter_accuracy.csv", index=False)
print("✓ graph_data_scatter_accuracy.csv")
print(f"  Rows: {len(scatter_data)} (4000 records)")


# ==============================================================================
# 6. DATA UNTUK GRAFIK BOXPLOT ERROR PER KELOMPOK
# ==============================================================================
"""
Format: method | group_theta | error (untuk setiap siswa)
Digunakan untuk: viz_error_boxplot.png

Ini adalah data "long format" yang cocok untuk boxplot
"""

boxplot_data = df_results[['method', 'group_theta', 'theta', 'true_score', 'score']].copy()
boxplot_data['error'] = boxplot_data['score'] - boxplot_data['true_score']

boxplot_data.to_csv("graph_data_error_boxplot.csv", index=False)
print("✓ graph_data_error_boxplot.csv")
print(f"  Rows: {len(boxplot_data)} (4000 records)")


# ==============================================================================
# 7. DATA AGREGAT ERROR PER KELOMPOK (untuk tabel summary)
# ==============================================================================
"""
Format: method | group_theta | mean_error | median_error | q25 | q75 | iqr
Digunakan untuk: membuat tabel statistik error
"""

error_summary_data = []

for method in methods:
    subset = df_results[df_results['method'] == method].copy()
    subset['error'] = subset['score'] - subset['true_score']

    for group in labels_5:
        group_subset = subset[subset['group_theta'] == group]

        errors = group_subset['error']

        error_summary_data.append({
            'method': method,
            'group_theta': group,
            'mean_error': errors.mean(),
            'median_error': errors.median(),
            'std_error': errors.std(),
            'q25': errors.quantile(0.25),
            'q75': errors.quantile(0.75),
            'iqr': errors.quantile(0.75) - errors.quantile(0.25),
            'min_error': errors.min(),
            'max_error': errors.max(),
            'n_students': len(errors)
        })

df_error_summary = pd.DataFrame(error_summary_data)
df_error_summary.to_csv("graph_data_error_summary.csv", index=False)
print("✓ graph_data_error_summary.csv")
print(f"  Rows: {len(df_error_summary)} (4 methods × 5 groups)")


# ==============================================================================
# 8. DATA UNTUK COMPARISON BAR CHART (Metrik Global)
# ==============================================================================
"""
Format: method | metric | value
Digunakan untuk: membuat comparison chart berbagai metrik

Ini adalah pivot dari summary_global.csv ke long format
"""

comparison_data = []

for _, row in df_summary.iterrows():
    comparison_data.append({'method': row['method'], 'metric': 'Correlation', 'value': row['correlation']})
    comparison_data.append({'method': row['method'], 'metric': 'RMSE', 'value': row['rmse']})
    comparison_data.append({'method': row['method'], 'metric': 'Bias', 'value': row['bias']})
    comparison_data.append({'method': row['method'], 'metric': 'Avg Items', 'value': row['avg_items']})

df_comparison = pd.DataFrame(comparison_data)
df_comparison.to_csv("graph_data_comparison_metrics.csv", index=False)
print("✓ graph_data_comparison_metrics.csv")
print(f"  Rows: {len(df_comparison)} (4 methods × 4 metrics)")


# ==============================================================================
# 9. DATA DISTRIBUSI DIFFICULTY PER KELOMPOK THETA
# ==============================================================================
"""
Format: method | group_theta | D_level | count | percentage
Bonus: untuk analisis tambahan difficulty distribution
"""

difficulty_theta_data = []

for method in methods:
    subset = df_results[df_results['method'] == method]

    for group in labels_5:
        group_subset = subset[subset['group_theta'] == group]

        # Expand history
        expanded = []
        for _, row in group_subset.iterrows():
            for h in row['history']:
                expanded.append(h['D'])

        total = len(expanded)

        for d in ['D1', 'D2', 'D3']:
            count = expanded.count(d)
            pct = (count / total * 100) if total > 0 else 0

            difficulty_theta_data.append({
                'method': method,
                'group_theta': group,
                'D_level': d,
                'count': count,
                'percentage': pct
            })

df_difficulty_theta = pd.DataFrame(difficulty_theta_data)
df_difficulty_theta.to_csv("graph_data_difficulty_per_theta.csv", index=False)
print("✓ graph_data_difficulty_per_theta.csv")
print(f"  Rows: {len(df_difficulty_theta)} (4 methods × 5 groups × 3 D-levels)")


# ==============================================================================
# 10. DATA CONVERGENCE PATTERN (untuk 5 siswa terpilih)
# ==============================================================================
"""
Format: student_id | method | q_no | theta_est | se | es | cumulative_correct
Digunakan untuk: melihat pola konvergensi estimasi skor
"""

convergence_data = []

for sid in selected_students:
    student_records = df_results[df_results['student_id'] == sid]

    for _, record in student_records.iterrows():
        method = record['method']
        cumulative_correct = 0

        for h in record['history']:
            cumulative_correct += h['is_correct']

            convergence_data.append({
                'student_id': sid,
                'theta': record['theta'],
                'true_score': record['true_score'],
                'method': method,
                'q_no': h['q_no'],
                'C_level': h['C'],
                'D_level': h['D'],
                'is_correct': h['is_correct'],
                'cumulative_correct': cumulative_correct,
                'theta_est': h.get('theta_est', None),
                'se': h.get('se', None),
                'es': h.get('es', None)
            })

df_convergence = pd.DataFrame(convergence_data)
df_convergence.to_csv("graph_data_convergence_5students.csv", index=False)
print("✓ graph_data_convergence_5students.csv")
print(f"  Rows: {len(df_convergence)} (5 students × 4 methods × avg items)")


# ==============================================================================
# SUMMARY: SEMUA FILE CSV YANG DIHASILKAN
# ==============================================================================

print("\n" + "=" * 80)
print("DAFTAR LENGKAP FILE CSV")
print("=" * 80)

csv_files = {
    "DATA UTAMA": [
        "results_full.csv - Hasil lengkap semua siswa (tanpa detail history)",
        "info_siswa.csv - Info theta dan clustering siswa",
        "detail_5_siswa.csv - Detail item-by-item 5 siswa terpilih"
    ],
    "SUMMARY TABLES": [
        "summary_global.csv - Metrik global per metode",
        "summary_per_kelompok.csv - Metrik per kelompok theta",
        "distribusi_kognitif.csv - Distribusi C1-C6 global",
        "distribusi_difficulty.csv - Distribusi D1-D3 global"
    ],
    "DATA UNTUK GRAFIK": [
        "graph_data_cognitive_per_theta.csv - Distribusi kognitif per kelompok theta",
        "graph_data_cognitive_per_score.csv - Distribusi kognitif per kelompok skor",
        "graph_data_items_per_theta.csv - Avg items per kelompok theta",
        "graph_data_items_per_score.csv - Avg items per kelompok skor",
        "graph_data_scatter_accuracy.csv - Data scatter plot (simplified)",
        "graph_data_error_boxplot.csv - Data untuk boxplot error",
        "graph_data_error_summary.csv - Summary statistik error",
        "graph_data_comparison_metrics.csv - Comparison metrik global",
        "graph_data_difficulty_per_theta.csv - Distribusi difficulty per kelompok",
        "graph_data_convergence_5students.csv - Pola konvergensi 5 siswa"
    ]
}

for category, files in csv_files.items():
    print(f"\n{category}:")
    for file in files:
        print(f"  ✓ {file}")

print("\n" + "=" * 80)
print("TOTAL: 17 FILE CSV SIAP UNTUK ANALISIS DI EXCEL/GOOGLE SHEETS!")
print("=" * 80)

SIMULASI PERBANDINGAN 4 METODE ASESMEN ADAPTIF

Metode yang dibandingkan:
1. CAT dengan EAP (Expected A Posteriori)
2. CAT dengan MLE (Maximum Likelihood Estimation)
3. Rule-Based dengan Prior (Bayesian Scoring)
4. Rule-Based tanpa Prior (Direct Weighted Scoring)

✓ Konfigurasi parameter telah dimuat

✓ Bank soal berhasil dibuat: 1008 items
  - Struktur: 6 Cognitive × 3 Difficulty × 56 items = 1008
  - Range difficulty (b): [-2.19, 1.79]
  - Range discrimination (a): [0.80, 1.50]

✓ Fungsi IRT telah didefinisikan
✓ Fungsi EAP telah didefinisikan
✓ Fungsi MLE telah didefinisikan
✓ Fungsi item selection telah didefinisikan
✓ Fungsi CAT-EAP telah didefinisikan
✓ Fungsi CAT-MLE telah didefinisikan
✓ Fungsi routing logic telah didefinisikan
✓ Fungsi Rule-Based dengan Prior telah didefinisikan
✓ Fungsi Rule-Based tanpa Prior telah didefinisikan

✓ Generated 1000 siswa
  Theta range: [-3.02, 3.14]
  Theta mean: 0.01, SD: 0.97

MENJALANKAN SIMULASI...
Progress: 250/1000 siswa (25.0%)
Progress: